In [39]:
pip install tensorboardx

Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
from datetime import datetime

def Write_to_file(Date, net_worth, filename='{}.txt'.format(datetime.now().strftime("%Y-%m-%d %H:%M:%S"))):
    for i in net_worth: 
        Date += " {}".format(i)
    #print(Date)
    if not os.path.exists('logs'):
        os.makedirs('logs')
    file = open("logs/"+filename, 'a+')
    file.write(Date+"\n")
    file.close()

In [2]:
import pandas as pd
from collections import deque
import matplotlib.pyplot as plt
from mplfinance.original_flavor import candlestick_ohlc
import matplotlib.dates as mpl_dates
from datetime import datetime
import os
import cv2
import numpy as np

class TradingGraph:
    # A crypto trading visualization using matplotlib made to render custom prices which come in following way:
    # Date, Open, High, Low, Close, Volume, net_worth, trades
    # call render every step
    def __init__(self, Render_range):
        self.Volume = deque(maxlen=Render_range)
        self.net_worth = deque(maxlen=Render_range)
        self.render_data = deque(maxlen=Render_range)
        self.Render_range = Render_range

        # We are using the style ‘ggplot’
        plt.style.use('ggplot')
        # close all plots if there are open
        plt.close('all')
        # figsize attribute allows us to specify the width and height of a figure in unit inches
        self.fig = plt.figure(figsize=(16,8)) 

        # Create top subplot for price axis
        self.ax1 = plt.subplot2grid((6,1), (0,0), rowspan=5, colspan=1)
        
        # Create bottom subplot for volume which shares its x-axis
        self.ax2 = plt.subplot2grid((6,1), (5,0), rowspan=1, colspan=1, sharex=self.ax1)
        
        # Create a new axis for net worth which shares its x-axis with price
        self.ax3 = self.ax1.twinx()

        # Formatting Date
        self.date_format = mpl_dates.DateFormatter('%d-%m-%Y')
        #self.date_format = mpl_dates.DateFormatter('%d-%m-%Y')
        
        # Add paddings to make graph easier to view
        #plt.subplots_adjust(left=0.07, bottom=-0.1, right=0.93, top=0.97, wspace=0, hspace=0)

        # we need to set layers
        self.ax2.set_xlabel('Date')
        self.ax1.set_ylabel('Price')
        self.ax3.set_ylabel('Balance')

        # I use tight_layout to replace plt.subplots_adjust
        self.fig.tight_layout()

        # Show the graph with matplotlib
        plt.show()
        
    def render(self, Date, Open, High, Low, Close, Volume, net_worth, trades):
        # append volume and net_worth to deque list
        self.Volume.append(Volume)
        self.net_worth.append(net_worth)

        # before appending to deque list, need to convert Date to special format
        Date = mpl_dates.date2num([pd.to_datetime(Date)])[0]
        self.render_data.append([Date, Open, High, Low, Close])
        
        # Clear the frame rendered last step
        self.ax1.clear()
        candlestick_ohlc(self.ax1, self.render_data, width=0.8/24, colorup='green', colordown='red', alpha=0.8)

        # Put all dates to one list and fill ax2 sublot with volume
        Date_Render_range = [i[0] for i in self.render_data]
        self.ax2.clear()
        self.ax2.fill_between(Date_Render_range, self.Volume, 0)

        # draw our net_worth graph on ax3 (shared with ax1) subplot
        self.ax3.clear()
        self.ax3.plot(Date_Render_range, self.net_worth, color="blue")
        
        # beautify the x-labels (Our Date format)
        self.ax1.xaxis.set_major_formatter(self.date_format)
        self.fig.autofmt_xdate()

        # sort sell and buy orders, put arrows in appropiate order positions
        for trade in trades:
            trade_date = mpl_dates.date2num([pd.to_datetime(trade['Date'])])[0]
            if trade_date in Date_Render_range:
                if trade['type'] == 'buy':
                    high_low = trade['Low']-10
                    self.ax1.scatter(trade_date, high_low, c='green', label='green', s = 120, edgecolors='none', marker="^")
                else:
                    high_low = trade['High']+10
                    self.ax1.scatter(trade_date, high_low, c='red', label='red', s = 120, edgecolors='none', marker="v")

        # we need to set layers every step, because we are clearing subplots every step
        self.ax2.set_xlabel('Date')
        self.ax1.set_ylabel('Price')
        self.ax3.set_ylabel('Balance')

        # I use tight_layout to replace plt.subplots_adjust
        self.fig.tight_layout()

        """Display image with matplotlib - interrupting other tasks"""
        # Show the graph without blocking the rest of the program
        #plt.show(block=False)
        # Necessary to view frames before they are unrendered
        #plt.pause(0.001)

        """Display image with OpenCV - no interruption"""
        # redraw the canvas
        self.fig.canvas.draw()
        # convert canvas to image
        img = np.fromstring(self.fig.canvas.tostring_rgb(), dtype=np.uint8, sep='')
        img  = img.reshape(self.fig.canvas.get_width_height()[::-1] + (3,))

        # img is rgb, convert to opencv's default bgr
        image = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

        # display image with OpenCV or any operation you like
        cv2.imshow("Bitcoin trading bot",image)

        if cv2.waitKey(25) & 0xFF == ord("q"):
            cv2.destroyAllWindows()
            return

In [19]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten
from tensorflow.keras import backend as K
from tensorboardX import SummaryWriter
from tensorflow.keras.optimizers.legacy import Adam, RMSprop
#tf.config.experimental_run_functions_eagerly(True) # used for debuging and development
tf.compat.v1.disable_eager_execution() # usually using this for fastest performance

gpus = tf.config.experimental.list_physical_devices('GPU')
if len(gpus) > 0:
    print(f'GPUs {gpus}')
    try: tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError: pass

class Actor_Model:
    def __init__(self, input_shape, action_space, lr, optimizer):
        X_input = Input(input_shape)
        self.action_space = action_space

        X = Flatten(input_shape=input_shape)(X_input)
        X = Dense(512, activation="relu")(X)
        X = Dense(256, activation="relu")(X)
        X = Dense(64, activation="relu")(X)
        output = Dense(self.action_space, activation="softmax")(X)

        self.Actor = Model(inputs = X_input, outputs = output)
        self.Actor.compile(loss=self.ppo_loss, optimizer=optimizer(lr=lr))

    def ppo_loss(self, y_true, y_pred):
        # Defined in https://arxiv.org/abs/1707.06347
        advantages, prediction_picks, actions = y_true[:, :1], y_true[:, 1:1+self.action_space], y_true[:, 1+self.action_space:]
        LOSS_CLIPPING = 0.2
        ENTROPY_LOSS = 0.001
        
        prob = actions * y_pred
        old_prob = actions * prediction_picks

        prob = K.clip(prob, 1e-10, 1.0)
        old_prob = K.clip(old_prob, 1e-10, 1.0)

        ratio = K.exp(K.log(prob) - K.log(old_prob))
        
        p1 = ratio * advantages
        p2 = K.clip(ratio, min_value=1 - LOSS_CLIPPING, max_value=1 + LOSS_CLIPPING) * advantages

        actor_loss = -K.mean(K.minimum(p1, p2))

        entropy = -(y_pred * K.log(y_pred + 1e-10))
        entropy = ENTROPY_LOSS * K.mean(entropy)
        
        total_loss = actor_loss - entropy

        return total_loss

    def predict(self, state):
        return self.Actor.predict(state)

class Critic_Model:
    def __init__(self, input_shape, action_space, lr, optimizer):
        X_input = Input(input_shape)

        V = Flatten(input_shape=input_shape)(X_input)
        V = Dense(512, activation="relu")(V)
        V = Dense(256, activation="relu")(V)
        V = Dense(64, activation="relu")(V)
        value = Dense(1, activation=None)(V)

        self.Critic = Model(inputs=X_input, outputs = value)
        self.Critic.compile(loss=self.critic_PPO2_loss, optimizer=optimizer(lr=lr))

    def critic_PPO2_loss(self, y_true, y_pred):
        value_loss = K.mean((y_true - y_pred) ** 2) # standard PPO loss
        return value_loss

    def predict(self, state):
        return self.Critic.predict([state, np.zeros((state.shape[0], 1))])

In [4]:
pip install model

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement model (from versions: none)
ERROR: No matching distribution found for model


In [28]:
import pandas as pd
import numpy as np
import random
from collections import deque
import copy

class CustomEnv:
    # A custom Bitcoin trading environment
    def __init__(self, df, initial_balance=1000, lookback_window_size=50, Render_range = 100):
        # Define action space and state size and other custom parameters
        self.df = df.dropna().reset_index()
        self.df_total_steps = len(self.df)-1
        self.initial_balance = initial_balance
        self.lookback_window_size = lookback_window_size
        self.Render_range = Render_range

        # Action space from 0 to 3, 0 is hold, 1 is buy, 2 is sell
        self.action_space = np.array([0, 1, 2])

        # Orders history contains the balance, net_worth, crypto_bought, crypto_sold, crypto_held values for the last lookback_window_size steps
        self.orders_history = deque(maxlen=self.lookback_window_size)
        
        # Market history contains the OHCL values for the last lookback_window_size prices
        self.market_history = deque(maxlen=self.lookback_window_size)

        # State size contains Market+Orders history for the last lookback_window_size steps
        self.state_size = (self.lookback_window_size, 10)
        # Neural Networks part bellow
        self.lr = 0.0001
        self.epochs = 1
        self.normalize_value = 100000
        self.optimizer = Adam
        self
        # Create Actor-Critic network model
        self.Actor = Actor_Model(input_shape=self.state_size, action_space = self.action_space.shape[0], lr=self.lr, optimizer = self.optimizer)
        self.Critic = Critic_Model(input_shape=self.state_size, action_space = self.action_space.shape[0], lr=self.lr, optimizer = self.optimizer)

    # Reset the state of the environment to an initial state
    def reset(self, env_steps_size = 0):
        self.balance = self.initial_balance
        self.net_worth = self.initial_balance
        self.prev_net_worth = self.initial_balance
        self.crypto_held = 0
        self.crypto_sold = 0
        self.crypto_bought = 0
        self.episode_orders = 0 # test
        if env_steps_size > 0: # used for training dataset
            self.start_step = random.randint(self.lookback_window_size, self.df_total_steps - env_steps_size)
            self.end_step = self.start_step + env_steps_size
            self.episode_orders += 1
        else: # used for testing dataset
            self.start_step = self.lookback_window_size
            self.end_step = self.df_total_steps
            self.episode_orders += 1

        self.current_step = self.start_step

        for i in reversed(range(self.lookback_window_size)):
            current_step = self.current_step - i
            self.orders_history.append([self.balance, self.net_worth, self.crypto_bought, self.crypto_sold, self.crypto_held])
            self.market_history.append([self.df.loc[current_step, 'open'],
                                    self.df.loc[current_step, 'high'],
                                    self.df.loc[current_step, 'low'],
                                    self.df.loc[current_step, 'close'],
                                    self.df.loc[current_step, 'volume']
                                    ])

        state = np.concatenate((self.market_history, self.orders_history), axis=1)
        return state
    
    def _next_observation(self):
        self.market_history.append([self.df.loc[self.current_step, 'open'],
                                    self.df.loc[self.current_step, 'high'],
                                    self.df.loc[self.current_step, 'low'],
                                    self.df.loc[self.current_step, 'close'],
                                    self.df.loc[self.current_step, 'volume']
                                    ])
        obs = np.concatenate((self.market_history, self.orders_history), axis=1)
        return obs
    
    def step(self, action):
        self.crypto_bought = 0
        self.crypto_sold = 0
        self.current_step += 1

        # Set the current price to a random price between open and close
        current_price = random.uniform(
            self.df.loc[self.current_step, 'open'],
            self.df.loc[self.current_step, 'close'])

        if action == 0: # Hold
            pass

        elif action == 1 and self.balance > 0:
            # Buy with 100% of current balance
            self.crypto_bought = self.balance / current_price
            self.balance -= self.crypto_bought * current_price
            self.crypto_held += self.crypto_bought

        elif action == 2 and self.crypto_held>0:
            # Sell 100% of current crypto held
            self.crypto_sold = self.crypto_held
            self.balance += self.crypto_sold * current_price
            self.crypto_held -= self.crypto_sold

        self.prev_net_worth = self.net_worth
        self.net_worth = self.balance + self.crypto_held * current_price

        self.orders_history.append([self.balance, self.net_worth, self.crypto_bought, self.crypto_sold, self.crypto_held])

        # Calculate reward
        reward = self.net_worth - self.prev_net_worth

        if self.net_worth <= self.initial_balance/2:
            done = True
        else:
            done = False

        obs = self._next_observation()

        return obs, reward, done
    
    def create_writer(self):
        self.replay_count = 0
        self.writer = SummaryWriter(comment="Trader")

    def render(self, visualize = False):
        #print(f'Step: {self.current_step}, Net Worth: {self.net_worth}')
        if visualize:
            Open = self.df.loc[self.current_step, 'open']
            Close = self.df.loc[self.current_step, 'close']
            High = self.df.loc[self.current_step, 'high']
            Low = self.df.loc[self.current_step, 'low']
            Volume = self.df.loc[self.current_step, 'volume']

            # Render the environment to the screen
            self.visualization.render(Date, Open, High, Low, Close, Volume, self.net_worth, self.trades)
    
    def get_gaes(self, rewards, dones, values, next_values, gamma = 0.99, lamda = 0.95, normalize=True):
        deltas = [r + gamma * (1 - d) * nv - v for r, d, nv, v in zip(rewards, dones, next_values, values)]
        deltas = np.stack(deltas)
        gaes = copy.deepcopy(deltas)
        for t in reversed(range(len(deltas) - 1)):
            gaes[t] = gaes[t] + (1 - dones[t]) * gamma * lamda * gaes[t + 1]

        target = gaes + values
        if normalize:
            gaes = (gaes - gaes.mean()) / (gaes.std() + 1e-8)
        return np.vstack(gaes), np.vstack(target)
    
    def replay(self, states, actions, rewards, predictions, dones, next_states):
        # reshape memory to appropriate shape for training
        states = np.vstack(states)
        next_states = np.vstack(next_states)
        actions = np.vstack(actions)
        predictions = np.vstack(predictions)

        # Compute discounted rewards
        #discounted_r = np.vstack(self.discount_rewards(rewards))

        # Get Critic network predictions 
        values = self.Critic.predict(states)
        next_values = self.Critic.predict(next_states)
        # Compute advantages
        #advantages = discounted_r - values
        advantages, target = self.get_gaes(rewards, dones, np.squeeze(values), np.squeeze(next_values))
        '''
        pylab.plot(target,'-')
        pylab.plot(advantages,'.')
        ax=pylab.gca()
        ax.grid(True)
        pylab.show()
        '''
        # stack everything to numpy array
        y_true = np.hstack([advantages, predictions, actions])
        
        # training Actor and Critic networks
        a_loss = self.Actor.Actor.fit(states, y_true, epochs=self.epochs, verbose=0, shuffle=True)
        c_loss = self.Critic.Critic.fit(states, target, epochs=self.epochs, verbose=0, shuffle=True)

        self.writer.add_scalar('Data/actor_loss_per_replay', np.sum(a_loss.history['loss']), self.replay_count)
        self.writer.add_scalar('Data/critic_loss_per_replay', np.sum(c_loss.history['loss']), self.replay_count)
        self.replay_count += 1
        
    def act(self, state):
        # Use the network to predict the next action to take, using the model
        prediction = self.Actor.predict(np.expand_dims(state, axis=0))[0]
        action = np.random.choice(self.action_space, p=prediction)
        return action, prediction

    def save(self, name="Crypto_trader"):
        # save keras model weights
        self.Actor.Actor.save_weights(f"{name}_Actor.h5")
        self.Critic.Critic.save_weights(f"{name}_Critic.h5")

    def load(self, name="Crypto_trader"):
        # load keras model weights
        self.Actor.Actor.load_weights(f"{name}_Actor.h5")
        self.Critic.Critic.load_weights(f"{name}_Critic.h5")

In [21]:
def train_agent(env, visualize=False, train_episodes = 50, training_batch_size=500):
    env.create_writer() # create TensorBoard writer
    total_average = deque(maxlen=100) # save recent 100 episodes net worth
    best_average = 0 # used to track best average net worth
    for episode in range(train_episodes):
        state = env.reset(env_steps_size = training_batch_size)

        states, actions, rewards, predictions, dones, next_states = [], [], [], [], [], []
        for t in range(training_batch_size):
            env.render(visualize)
            action, prediction = env.act(state)
            next_state, reward, done = env.step(action)
            states.append(np.expand_dims(state, axis=0))
            next_states.append(np.expand_dims(next_state, axis=0))
            action_onehot = np.zeros(3)
            action_onehot[action] = 1
            actions.append(action_onehot)
            rewards.append(reward)
            dones.append(done)
            predictions.append(prediction)
            state = next_state
            
        env.replay(states, actions, rewards, predictions, dones, next_states)
        total_average.append(env.net_worth)
        average = np.average(total_average)
        
        env.writer.add_scalar('Data/average net_worth', average, episode)
        env.writer.add_scalar('Data/episode_orders', env.episode_orders, episode)
        
        print("net worth {} {:.2f} {:.2f} {}".format(episode, env.net_worth, average, env.episode_orders))
        if episode > len(total_average):
            if best_average < average:
                best_average = average
                print("Saving model")
                env.save()

In [22]:
def test_agent(env, visualize=True, test_episodes=10):
    env.load() # load the model
    average_net_worth = 0
    for episode in range(test_episodes):
        state = env.reset()
        while True:
            env.render(visualize)
            action, prediction = env.act(state)
            state, reward, done = env.step(action)
            if env.current_step == env.end_step:
                average_net_worth += env.net_worth
                print("net_worth:", episode, env.net_worth, env.episode_orders)
                break
            
    print("average {} episodes agent net_worth: {}".format(test_episodes, average_net_worth/test_episodes))

In [23]:
from tvDatafeed import TvDatafeed, Interval

username = 'No_Sock'
password = '#TradeCore$23'

tv = TvDatafeed(username, password)

In [24]:
cdpr_data = tv.get_hist(symbol='CDR',exchange='GPW',interval=Interval.in_1_hour,n_bars=10000)

In [25]:
df = cdpr_data[['open','high','low','close','volume']]
lookback_window_size = 50
train_df = df[:-720-lookback_window_size]
test_df = df[-720-lookback_window_size:] # 30 days

In [29]:
train_env = CustomEnv(train_df,lookback_window_size=lookback_window_size)
test_env = CustomEnv(test_df,lookback_window_size=lookback_window_size)

C:\Users\ThinkPadX4\anaconda3\lib\site-packages\keras\optimizers\legacy\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
C:\Users\ThinkPadX4\anaconda3\lib\site-packages\keras\optimizers\legacy\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [30]:
train_agent(train_env, visualize=False, train_episodes=20000, training_batch_size=500)

C:\Users\ThinkPadX4\anaconda3\lib\site-packages\keras\engine\training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


net worth 0 1000.00 1000.00 1
net worth 1 1000.00 1000.00 1
net worth 2 1000.00 1000.00 1
net worth 3 1000.00 1000.00 1
net worth 4 1000.00 1000.00 1
net worth 5 1016.14 1002.69 1
net worth 6 1000.00 1002.31 1
net worth 7 1000.00 1002.02 1
net worth 8 1000.00 1001.79 1
net worth 9 1000.00 1001.61 1
net worth 10 1000.00 1001.47 1
net worth 11 1000.00 1001.35 1
net worth 12 1000.00 1001.24 1
net worth 13 1000.00 1001.15 1
net worth 14 1000.00 1001.08 1
net worth 15 1000.00 1001.01 1
net worth 16 1000.00 1000.95 1
net worth 17 1000.00 1000.90 1
net worth 18 1000.00 1000.85 1
net worth 19 1000.00 1000.81 1
net worth 20 1000.00 1000.77 1
net worth 21 1000.00 1000.73 1
net worth 22 1000.00 1000.70 1
net worth 23 1000.00 1000.67 1
net worth 24 1000.00 1000.65 1
net worth 25 1000.00 1000.62 1
net worth 26 1000.00 1000.60 1
net worth 27 1000.00 1000.58 1
net worth 28 1000.00 1000.56 1
net worth 29 1000.00 1000.54 1
net worth 30 1000.00 1000.52 1
net worth 31 1000.00 1000.50 1
net worth 32 1000.

net worth 260 1000.00 1000.00 1
net worth 261 1000.00 1000.00 1
net worth 262 1000.00 1000.00 1
net worth 263 1000.00 1000.00 1
net worth 264 1000.00 1000.00 1
net worth 265 1000.00 1000.00 1
net worth 266 1000.00 1000.00 1
net worth 267 1000.00 1000.00 1
net worth 268 1000.00 1000.00 1
net worth 269 1000.00 1000.00 1
net worth 270 1000.00 1000.00 1
net worth 271 1000.00 1000.00 1
net worth 272 1000.00 1000.00 1
net worth 273 1000.00 1000.00 1
net worth 274 1000.00 1000.00 1
net worth 275 1000.00 1000.00 1
net worth 276 1000.00 1000.00 1
net worth 277 1000.00 1000.00 1
net worth 278 1000.00 1000.00 1
net worth 279 1000.00 1000.00 1
net worth 280 1000.00 1000.00 1
net worth 281 1000.00 1000.00 1
net worth 282 1000.00 1000.00 1
net worth 283 1000.00 1000.00 1
net worth 284 1000.00 1000.00 1
net worth 285 1000.00 1000.00 1
net worth 286 1000.00 1000.00 1
net worth 287 1000.00 1000.00 1
net worth 288 1000.00 1000.00 1
net worth 289 1000.00 1000.00 1
net worth 290 1000.00 1000.00 1
net wort

net worth 517 1000.00 1000.00 1
net worth 518 1000.00 1000.00 1
net worth 519 1000.00 1000.00 1
net worth 520 1000.00 1000.00 1
net worth 521 1000.00 1000.00 1
net worth 522 1000.00 1000.00 1
net worth 523 1000.00 1000.00 1
net worth 524 1000.00 1000.00 1
net worth 525 1000.00 1000.00 1
net worth 526 1000.00 1000.00 1
net worth 527 1000.00 1000.00 1
net worth 528 1000.00 1000.00 1
net worth 529 1000.00 1000.00 1
net worth 530 1000.00 1000.00 1
net worth 531 1000.00 1000.00 1
net worth 532 1000.00 1000.00 1
net worth 533 1000.00 1000.00 1
net worth 534 1000.00 1000.00 1
net worth 535 1000.00 1000.00 1
net worth 536 1000.00 1000.00 1
net worth 537 1000.00 1000.00 1
net worth 538 1000.00 1000.00 1
net worth 539 1000.00 1000.00 1
net worth 540 1000.00 1000.00 1
net worth 541 1000.00 1000.00 1
net worth 542 1000.00 1000.00 1
net worth 543 1000.00 1000.00 1
net worth 544 1000.00 1000.00 1
net worth 545 1000.00 1000.00 1
net worth 546 1000.00 1000.00 1
net worth 547 1000.00 1000.00 1
net wort

net worth 774 1000.00 1000.00 1
net worth 775 1000.00 1000.00 1
net worth 776 1000.00 1000.00 1
net worth 777 1000.00 1000.00 1
net worth 778 1000.00 1000.00 1
net worth 779 1000.00 1000.00 1
net worth 780 1000.00 1000.00 1
net worth 781 1000.00 1000.00 1
net worth 782 1000.00 1000.00 1
net worth 783 1000.00 1000.00 1
net worth 784 1000.00 1000.00 1
net worth 785 1000.00 1000.00 1
net worth 786 1000.00 1000.00 1
net worth 787 1000.00 1000.00 1
net worth 788 1000.00 1000.00 1
net worth 789 1000.00 1000.00 1
net worth 790 1000.00 1000.00 1
net worth 791 1000.00 1000.00 1
net worth 792 1000.00 1000.00 1
net worth 793 1000.00 1000.00 1
net worth 794 1000.00 1000.00 1
net worth 795 1000.00 1000.00 1
net worth 796 1000.00 1000.00 1
net worth 797 1000.00 1000.00 1
net worth 798 1000.00 1000.00 1
net worth 799 1000.00 1000.00 1
net worth 800 1000.00 1000.00 1
net worth 801 1000.00 1000.00 1
net worth 802 1000.00 1000.00 1
net worth 803 1000.00 1000.00 1
net worth 804 1000.00 1000.00 1
net wort

...

net worth 1279 1000.00 1000.00 1
net worth 1280 1000.00 1000.00 1
net worth 1281 1000.00 1000.00 1
net worth 1282 1000.00 1000.00 1
net worth 1283 1000.00 1000.00 1
net worth 1284 1000.00 1000.00 1
net worth 1285 1000.00 1000.00 1
net worth 1286 1000.00 1000.00 1
net worth 1287 1000.00 1000.00 1
net worth 1288 1000.00 1000.00 1
net worth 1289 1000.00 1000.00 1
net worth 1290 1000.00 1000.00 1
net worth 1291 1000.00 1000.00 1
net worth 1292 1000.00 1000.00 1
net worth 1293 1000.00 1000.00 1
net worth 1294 1000.00 1000.00 1
net worth 1295 1000.00 1000.00 1
net worth 1296 1000.00 1000.00 1
net worth 1297 1000.00 1000.00 1
net worth 1298 1000.00 1000.00 1
net worth 1299 1000.00 1000.00 1
net worth 1300 1000.00 1000.00 1
net worth 1301 1000.00 1000.00 1
net worth 1302 1000.00 1000.00 1
net worth 1303 1000.00 1000.00 1
net worth 1304 1000.00 1000.00 1
net worth 1305 1000.00 1000.00 1
net worth 1306 1000.00 1000.00 1
net worth 1307 1000.00 1000.00 1
net worth 1308 1000.00 1000.00 1
net worth 

net worth 1528 1000.00 1000.00 1
net worth 1529 1000.00 1000.00 1
net worth 1530 1000.00 1000.00 1
net worth 1531 1000.00 1000.00 1
net worth 1532 1000.00 1000.00 1
net worth 1533 1000.00 1000.00 1
net worth 1534 1000.00 1000.00 1
net worth 1535 1000.00 1000.00 1
net worth 1536 1000.00 1000.00 1
net worth 1537 1000.00 1000.00 1
net worth 1538 1000.00 1000.00 1
net worth 1539 1000.00 1000.00 1
net worth 1540 1000.00 1000.00 1
net worth 1541 1000.00 1000.00 1
net worth 1542 1000.00 1000.00 1
net worth 1543 1000.00 1000.00 1
net worth 1544 1000.00 1000.00 1
net worth 1545 1000.00 1000.00 1
net worth 1546 1000.00 1000.00 1
net worth 1547 1000.00 1000.00 1
net worth 1548 1000.00 1000.00 1
net worth 1549 1000.00 1000.00 1
net worth 1550 1000.00 1000.00 1
net worth 1551 1000.00 1000.00 1
net worth 1552 1000.00 1000.00 1
net worth 1553 1000.00 1000.00 1
net worth 1554 1000.00 1000.00 1
net worth 1555 1000.00 1000.00 1
net worth 1556 1000.00 1000.00 1
net worth 1557 1000.00 1000.00 1
net worth 

net worth 1777 1000.00 1000.00 1
net worth 1778 1000.00 1000.00 1
net worth 1779 1000.00 1000.00 1
net worth 1780 1000.00 1000.00 1
net worth 1781 1000.00 1000.00 1
net worth 1782 1000.00 1000.00 1
net worth 1783 1000.00 1000.00 1
net worth 1784 1000.00 1000.00 1
net worth 1785 1000.00 1000.00 1
net worth 1786 1000.00 1000.00 1
net worth 1787 1000.00 1000.00 1
net worth 1788 1000.00 1000.00 1
net worth 1789 1000.00 1000.00 1
net worth 1790 1000.00 1000.00 1
net worth 1791 1000.00 1000.00 1
net worth 1792 1000.00 1000.00 1
net worth 1793 1000.00 1000.00 1
net worth 1794 1000.00 1000.00 1
net worth 1795 1000.00 1000.00 1
net worth 1796 1000.00 1000.00 1
net worth 1797 1000.00 1000.00 1
net worth 1798 1000.00 1000.00 1
net worth 1799 1000.00 1000.00 1
net worth 1800 1000.00 1000.00 1
net worth 1801 1000.00 1000.00 1
net worth 1802 1000.00 1000.00 1
net worth 1803 1000.00 1000.00 1
net worth 1804 1000.00 1000.00 1
net worth 1805 1000.00 1000.00 1
net worth 1806 1000.00 1000.00 1
net worth 

net worth 2026 1000.00 1000.00 1
net worth 2027 1000.00 1000.00 1
net worth 2028 1000.00 1000.00 1
net worth 2029 1000.00 1000.00 1
net worth 2030 1000.00 1000.00 1
net worth 2031 1000.00 1000.00 1
net worth 2032 1000.00 1000.00 1
net worth 2033 1000.00 1000.00 1
net worth 2034 1000.00 1000.00 1
net worth 2035 1000.00 1000.00 1
net worth 2036 1000.00 1000.00 1
net worth 2037 1000.00 1000.00 1
net worth 2038 1000.00 1000.00 1
net worth 2039 1000.00 1000.00 1
net worth 2040 1000.00 1000.00 1
net worth 2041 1000.00 1000.00 1
net worth 2042 1000.00 1000.00 1
net worth 2043 1000.00 1000.00 1
net worth 2044 1000.00 1000.00 1
net worth 2045 1000.00 1000.00 1
net worth 2046 1000.00 1000.00 1
net worth 2047 1000.00 1000.00 1
net worth 2048 1000.00 1000.00 1
net worth 2049 1000.00 1000.00 1
net worth 2050 1000.00 1000.00 1
net worth 2051 1000.00 1000.00 1
net worth 2052 1000.00 1000.00 1
net worth 2053 1000.00 1000.00 1
net worth 2054 1000.00 1000.00 1
net worth 2055 1000.00 1000.00 1
net worth 

KeyboardInterrupt: 